# Science under the Stars Vitessce Vizualization

Directory Structure:
```sh
vitessce_suts/
├── data/
│   ├── data.fcs
│   └── embedded_data.zarr *
└── scripts/
    ├── process_fcs.ipynb
    └── vitessce.ipynb *
```


In [ ]:
from pathlib import Path

import numpy as np
from matplotlib.colors import to_rgb
from vitessce import (
    AnnDataWrapper,
    VitessceConfig,
)
from vitessce import Component as cm
from vitessce import CoordinationType as ct

## Create Vitessce Config

In [ ]:
cancer_cells = "Cancer Cells"
queen_bee_cells = "Queen Bee Cells"
healthy_cells = "Healthy Cells"

In [ ]:
def color_name_to_int(color_name):
    """Converts a color name to a list of integers representing the color in RGB

    Parameters
    ----------
    color_name : str
        The color to convert

    Returns
    -------
    list[int]
        The integer RGB values
    """
    rgb = to_rgb(color_name)
    return (np.array(rgb) * 255).astype(int).tolist()

In [ ]:
vit_adata_path = Path("../data/embedded_data.zarr")

In [ ]:
vc = VitessceConfig(schema_version="1.0.15", name="Science Under the Stars", description="The Dataset for the event")

dataset = vc.add_dataset(name="Dataset").add_object(
    AnnDataWrapper(
        adata_path=vit_adata_path,
        obs_embedding_paths=["obsm/X_umap_2D"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=["obs/cell_type_patient"],
        obs_set_names=["Cell Type"],
        obs_feature_matrix_path="X",
        coordination_values={
            "obsType": "cell",
            "featureType": "gene",
            "featureValueType": "expression",
        },
    )
)

In [ ]:
scatterplot_umap = vc.add_view(view_type=cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(view_type=cm.OBS_SETS, dataset=dataset)
markers = vc.add_view(view_type=cm.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(view_type=cm.HEATMAP, dataset=dataset)
cell_set_sizes = vc.add_view(view_type=cm.OBS_SET_SIZES, dataset=dataset)

vc.link_views_by_dict(
    views=[cell_sets, heatmap, cell_set_sizes],
    input_val={
        ct.OBS_COLOR_ENCODING: "cellSetSelection",
        ct.OBS_SET_SELECTION: [
            ["Cell Type", cancer_cells],
            ["Cell Type", queen_bee_cells],
            ["Cell Type", healthy_cells],
        ],
        ct.OBS_SET_COLOR: [
            {"path": ["Cell Type", healthy_cells], "color": color_name_to_int("xkcd:grey")},
            {"path": ["Cell Type", cancer_cells], "color": color_name_to_int("xkcd:green")},
            {"path": ["Cell Type", queen_bee_cells], "color": color_name_to_int("xkcd:red")},
        ],
    },
    meta=False,
)
vc.layout((scatterplot_umap | (cell_sets / markers)) / (heatmap | cell_set_sizes));

## Open the Web View

In [ ]:
vc.web_app()